In [ ]:
from ctmc import ContinuousTimeMarkovChain as MC
from ctmc import normal_generator, gamma_generator, uniform_generator, cyclic_generator, detailed_balance_generator

import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
#make 1 machines with 7 states, generated using the "cyclic" generator fo transition rates
machine = MC(S=6, N=1, generator=cyclic_generator)

In [ ]:
# the cornerstone is the rate matrix, note it is scaled so the maximum rate on the diagonal is -1, 
# this basically sets a uniform timescale for different models based on the state with the most outgoing activity
print(machine.R)

# this scaling is kept track of with the machine.scale attribute
machine.scale

In [ ]:
# find and the NESS state
ness = machine.get_ness()
# finds and the minimum entropy producing state
meps = machine.get_meps()
# returns a uniform state
unif = machine.get_uniform()


In [ ]:
# for states, we can grab a variety of values, like the entropy produciton rate
print('epr:')
print(machine.get_epr(ness), machine.get_epr(meps), machine.get_epr(unif))
#activity (which is the sum of all (positive) transition rates)
print('activity:')
print(machine.get_activity(ness), machine.get_activity(meps), machine.get_activity(unif))
# or probability current (which is the total amount of probability flowing between states, note it is zero for the NESS state)
print('current:')
print(machine.get_prob_current(ness), machine.get_prob_current(meps), machine.get_prob_current(unif))

In [ ]:
print(machine.get_prob_current(ness), machine.get_prob_current(meps), machine.get_prob_current(unif))

In [ ]:
# returns a random state drawn from a uniform distrbution over all states
random = machine.get_random_state()

# returns a "localized" state which is basically a guassian centered on a particular state, putting the states on a ring, default is random peak and variance
random_local = machine.get_local_state()
# you can put in manual arguments too, though theres an annoying input issue with variances for now
hardcoded_local_1 = machine.get_local_state(mu=2.6, sigma= np.array([.75]))
hardcoded_local_2 = machine.get_local_state(mu=2.6, sigma= np.array([1.5]))


fig, ax = plt.subplots(figsize=(10,6))
states = [random, random_local, hardcoded_local_1, hardcoded_local_2]
labels = ['rand','rand_loc', 'local_1', 'local_2']
for s,l in zip(states, labels):
    ax.plot(range(machine.S), s.T, label=l, marker='o')
ax.set_xlabel('state #');
ax.set_ylabel('p(state #)');
fig.legend()


In [ ]:
# and, we can evolve these states according to the rate matrix:
current_state = hardcoded_local_1
states = [current_state]
for i in range(20):
    current_state = machine.evolve_state(current_state, dt=.2)
    states.append(current_state)

fig, ax = plt.subplots()
ax.plot(np.squeeze(states), marker='o');
ax.set_xlabel('$t$');
ax.set_ylabel('$p_i(t)$');

In [ ]:
# all of these things also work with ensembles of machines, for example...

# make an ensemble of 100 machines with 50 states each, generated using the default uniform generator for transition rates
machines = MC(S=50, N=100)

In [ ]:
meps_100 = machines.get_meps()
meps_epr = machines.get_epr(meps_100)
meps_activity = machines.get_activity(meps_100)

random_100 = machines.get_random_state()
random_epr = machines.get_epr(random_100)
random_activity = machines.get_activity(random_100)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,7))
ax[0].scatter(range(100), meps_epr, label='meps')
ax[0].scatter(range(100), random_epr, label='rand')

ax[1].scatter(range(100), random_activity)
ax[1].scatter(range(100), meps_activity)

ax[0].set_title('entropy production rate')
ax[1].set_title('activity')

fig.legend()

In [ ]:
# example of doing a sweep over many different kind of generators with different S and N, and saving some of the data. The NESS calculating will become numeric if the matrices get
# too large to invert exactly
SS = [3, 6, 20, 100, 250]
trials = [200, 200, 200, 200, 200]

normal_output, uniform_output, gamma_output, cyclic_output, balanced_output = {}, {}, {}, {}, {}

for a in range(5):
    print(a)
    output = [normal_output, uniform_output, gamma_output, cyclic_output, balanced_output ][a]
    gen = [normal_generator, uniform_generator, gamma_generator, cyclic_generator, detailed_balance_generator][a]
    
    
    for s,trial in zip(SS,trials):
        p = MC(S=s, N=trial, generator=gen)
        #R = [ np.random.uniform(0,1,(S,S)) for i in range(trials)]
        #for p,r in zip(procs,R):
        #    p.set_rate_matrix(r, max_rate=1)   
        dct = {'ness':p.get_epr(p.get_ness()), 
               'meps':p.get_epr(p.get_meps()), 
               'unif':p.get_epr(p.get_uniform()),
               'rand':p.get_epr(p.get_random_state()),
               'local':p.get_epr(p.get_local_state()),
               'N':trial, 'scale':p.scale, 
               'nm_dkl':p.dkl(p.ness,p.meps),'mn_dkl':p.dkl(p.meps,p.ness)}
        
        
        output[f'{p.S}'] = dct


In [ ]:
# some example data calls
plt.scatter(cyclic_output['6']['meps'],cyclic_output['6']['local'],)